# P2SH MultiSig BIP322
## Role: Signer

## 1. Generate HD Key from mneumonic

In [1]:
from buidl.mnemonic import secure_mnemonic
from buidl.hd import HDPrivateKey

# mnemonic = secure_mnemonic()

mnemonic = 'army injury wrestle spin loyal skill acid word hobby prosper purity disorder impulse observe elevator perfect deer student juice reveal portion nice quarter banana'

hd_root = HDPrivateKey.from_mnemonic(mnemonic)
root_xfp = hd_root.fingerprint().hex()

In [2]:
hd_root.wif()

'L1zt9Rw7HrU7jaguMbVzhiX8ffuVkmMis5wLHddXYuHWYf8u8uRj'

## 2. Select Key to Sign With

In [5]:
bip32_path = "m/45'/0/0/6"

signing_privkey = hd_root.traverse(bip32_path)
signing_pubkey = signing_privkey.pub.point


## 3. Share Public Key with Creator

The public key to include in a p2sh redeem script must be communicated to the creator of a BIP322 signature through some communication channel. This channel is out of scope.

In this example we will copy a hex representation of the public key across to the [PSBT Creator notebook](http://localhost:8888/notebooks/p2sh_multisig_bip322/PSBT_creator.ipynb)

In [6]:
print("Copy the below line into step 3 of the creator notebook \n")
print(f"signer_2_pubkey = '{signing_pubkey.sec().hex()}'")

Copy the below line into step 3 of the creator notebook 

signer_2_pubkey = '02fae5c1f65f93bb2ee8e34cb0cbda5f0fc72d18d94511f7dbb5a827e3773edd92'


## 4. Recieve PSBT from Creator

Some out of band communication mechanism

In [1]:
#### Copy from PSBT_Creator notebook Step 8. ######

b64_psbt = 'cHNidP8BAD0AAAAAAc1x8do1+Xx//5kLaBOvxoX7QqCmliMoz3h6Bo2b+7VkAAAAAAAAAAAAAQAAAAAAAAAAAWoAAAAAAAEAeAAAAAAAAQEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAP////8iACA4E3j7MVlbnxgcxmUY+N1w459TuGihtX+nhDqV8uyuxQAAAAABAAAAAAAAAAAXqRTRdj7ago2+eug3TS1/tsM4GXCycocAAAAAAAEEaVIhAiiXxstvIpi3LgHjQSxwjb9zUT+Y5ycBQzUVwmDWP7yeIQL9UULWmbO/ryqkHlpdq2/43f4DMZBI1J42XQxjw2ZDCyEC+uXB9l+Tuy7o40ywy9pfD8ctGNlFEffbtagn43c+3ZJTriIGAiiXxstvIpi3LgHjQSxwjb9zUT+Y5ycBQzUVwmDWP7yeFHsF6hstAACAAAAAAAAAAAABAAAAAAA='

In [2]:
from buidl.psbt import PSBT
psbt_to_sign = PSBT.parse_base64(b64_psbt)

## 5. TODO: How does the signer Verify the Message that they are signing with this

I assume they must be given the means to construct the to_spend transaction themselves and verify that this is referenced as the only input to the tx_obj in the PSBT

## 6. Update PSBT with pubkey

In [9]:
from buidl.psbt import NamedHDPublicKey

pubkey_lookup = {}

named_hd_pubkey_obj = NamedHDPublicKey.from_hd_pub(
    child_hd_pub=signing_privkey.pub,
    xfp_hex=root_xfp,
    path=bip32_path,
)
# pubkey lookups needed for validation
pubkey_lookup[named_hd_pubkey_obj.sec()] = named_hd_pubkey_obj

In [10]:
# Note tx_lookup is set to None, because already been defined by creator
psbt_to_sign.update(None, pubkey_lookup)

## 7. Sign the PSBT

In [11]:
psbt_to_sign.sign(hd_root)

True

## 9. Serialize Signed PSBT and Return to Creator

Note: Communication channel out of band. We use copy and paste.

In [12]:
signer_2_psbt_b64 = psbt_to_sign.serialize_base64()
print("Copy this VVVV \n\n")
print(f"signer_2_psbt_b64 = '{signer_2_psbt_b64}'")

Copy this VVVV 


signer_2_psbt_b64 = 'cHNidP8BAD0AAAAAAc1x8do1+Xx//5kLaBOvxoX7QqCmliMoz3h6Bo2b+7VkAAAAAAAAAAAAAQAAAAAAAAAAAWoAAAAAAAEAeAAAAAAAAQEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAP////8iACA4E3j7MVlbnxgcxmUY+N1w459TuGihtX+nhDqV8uyuxQAAAAABAAAAAAAAAAAXqRTRdj7ago2+eug3TS1/tsM4GXCycocAAAAAACICAvrlwfZfk7su6ONMsMvaXw/HLRjZRRH327WoJ+N3Pt2SRzBEAiAfd5b9XIUOhe8Jqx3yrGDCPk0NWjTB5e8zqrIZcsJHlgIgTCNsRXp9LpFY/pCwo5wzC+UavX9KB0ILqlbh4fzv9iABAQRpUiECKJfGy28imLcuAeNBLHCNv3NRP5jnJwFDNRXCYNY/vJ4hAv1RQtaZs7+vKqQeWl2rb/jd/gMxkEjUnjZdDGPDZkMLIQL65cH2X5O7LujjTLDL2l8Pxy0Y2UUR99u1qCfjdz7dklOuIgYCKJfGy28imLcuAeNBLHCNv3NRP5jnJwFDNRXCYNY/vJ4UewXqGy0AAIAAAAAAAAAAAAEAAAAiBgL65cH2X5O7LujjTLDL2l8Pxy0Y2UUR99u1qCfjdz7dkhQAHQ2eLQAAgAAAAAAAAAAABgAAAAAA'
